# Школа глубокого обучения

<a href="https://mipt.ru/science/labs/laboratoriya-neyronnykh-sistem-i-glubokogo-obucheniya/"><img align="right" src="https://avatars1.githubusercontent.com/u/29918795?v=4&s=200" alt="DeepHackLab" style="position:relative;top:-40px;right:10px;height:100px;" /></a>



### Физтех-Школа Прикладной математики и информатики МФТИ 
### Лаборатория нейронных сетей и глубокого обучения (DeepHackLab)

*Илья Захаркин (ФИВТ МФТИ)*

## Работа с Word2Vec

Привет!

В данном ноутбуке мы посмотрим на простой пример работы с реализацией Word2Vec из библиотеки `gensim`, а также на работу другой модели эмбеддингов - GloVe.

### Word2Vec

In [22]:
from gensim.models import word2vec
from gensim import models

In [3]:
from gensim.models import word2vec
import logging

# настроим запись служебной информации в файл
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

Будем использовать датасет **Text8**. Загрузим корпус текстов с сайта http://mattmahoney.net/dc/text8.zip (30 Mb)

In [6]:
sentences = word2vec.Text8Corpus('./data/text8')

Обучение модели. По-умолчанию используется реализация **skip-gram** с окном контекста *window=5*:

In [26]:
# 3-5 минут
model = word2vec.Word2Vec(sentences, size=200)

Посмотрим, что модель умеет, а именно выведем самые близкие по смыслу слова, удовлетворяющие условиям `positive` и `negative` примеров (то есть слова, близкие к `positive`, но далёкие от `negative`). Параметр `topn` отвечает за то, сколько наиболее вероятных (по убыванию вероятности) слов выдавать:

In [13]:
# ожидаем увидеть "queen"
model.wv.most_similar(positive=['woman', 'king'], negative=['man'], topn=1)

[('queen', 0.6541075706481934)]

И снова посмотрим на пример:

In [16]:
# ожидаем "mother"
model.wv.most_similar(['girl', 'father'], ['boy'], topn=3)

[('mother', 0.7719833850860596),
 ('wife', 0.7053205966949463),
 ('grandmother', 0.7014738917350769)]

Больше примеров соотношения слов в векторном пространстве:

In [17]:
more_examples = ["he his she", 
                 "big bigger bad", 
                 "going went being"]

for example in more_examples:
    a, b, x = example.split()
    predicted = model.wv.most_similar([x, b], [a])[0][0]
    print("'%s' соотносится с '%s' также, как '%s' соотносится с '%s'" % (a, b, x, predicted))

'he' соотносится с 'his' также, как 'she' соотносится с 'her'
'big' соотносится с 'bigger' также, как 'bad' соотносится с 'worse'
'going' соотносится с 'went' также, как 'being' соотносится с 'was'


Модель также умеет находить слова, которые не подходят "по смыслу" другим словам из данной последовательности

In [19]:
# по порядку: "завтрак" "зерновой" "обед" "ланч"
model.wv.doesnt_match("breakfast cereal dinner lunch".split())

'cereal'

---

Так мы можем сохранить модель (матрицу весов (эмбеддингов)), чтобы потом можно было её переиспользовать, если нужно:

In [9]:
model.save('text8.model')

2018-03-07 02:49:22,021 : INFO : saving Word2Vec object under text8.model, separately None
2018-03-07 02:49:22,023 : INFO : not storing attribute syn0norm
2018-03-07 02:49:22,025 : INFO : storing np array 'syn0' to text8.model.wv.syn0.npy
2018-03-07 02:49:22,855 : INFO : not storing attribute cum_table
2018-03-07 02:49:22,856 : INFO : storing np array 'syn1neg' to text8.model.syn1neg.npy
2018-03-07 02:49:23,759 : INFO : saved text8.model


Можем указать и другой формат:

In [12]:
model.wv.save_word2vec_format('text8.model.bin', binary=True)

2018-03-07 02:49:43,596 : INFO : storing 71290x200 projection weights into text8.model.bin


---

In [20]:
# Здесь можно ещё поиграться с Word2Vec




### GloVe

*Примечание:*   
С установкой glove на Windows могут быть трудности (у автора они тоже возникли), но если получится установить, то следующий код должен отработать и вы увидите примеры работы GloVe:

In [ ]:
# !pip install glove

In [ ]:
import itertools
from gensim.models.word2vec import Text8Corpus

from glove import Corpus, Glove

Снова загрузим данные:

In [ ]:
sentences = list(itertools.islice(Text8Corpus('./data/text8'), None))

# Специальный класс для корпуса в GloVe
corpus = Corpus()
corpus.fit(sentences, window=10)

Инициализация класса.   
`num_components` - размерность эмбеддинга,  
`learning_rate` - вам должно быть уже знакомо ;) (скорость обучения)

In [ ]:
glove = Glove(no_components=100, learning_rate=0.05)

Обучение модели:

In [ ]:
glove.fit(corpus.matrix, epochs=30, no_threads=4, verbose=True)

Сохранение словаря в модель:

In [ ]:
glove.add_dictionary(corpus.dictionary)

Примеры:

In [ ]:
glove.most_similar('man')

In [ ]:
glove.most_similar('frog', number=10)

In [ ]:
glove.most_similar('girl', number=10)

In [ ]:
 glove.most_similar('car', number=10)

In [ ]:
glove.most_similar('queen', number=10)

In [ ]:
# Здесь можно ещё поиграться с GloVe




### Полезые ссылки

В интернете достаточно много примеров работы с Word2Vec. Вот некоторые из них:

* Официальный tutorial от создателей gensim: https://radimrehurek.com/gensim/models/word2vec.html
* Tutorial по построению мини-web-приложения на основе Word2Vec: https://rare-technologies.com/word2vec-tutorial/
* Визуалцизация Word2Vec-эмбеддингов с помощью Tensorboard (нужен установленная библиотека глубокого обучения **tensorflow** для запуска у себя на компьютере, или просто можно посмотреть, ак оно на видео): https://www.youtube.com/watch?time_continue=181&v=BkeQzJt0f5A